<a href="https://colab.research.google.com/github/R06880/Covid19-Detection-Using-CNN/blob/main/Covid19_Detection_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Convolutional Neural Network to predict Covid19/Normal X-Rays***

In [ ]:
# the images are in my google drive account, so we need to import drive

from google.colab import drive

In [11]:
# making the connection to Google drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
# import libraries
# we will preprocess the images using the preprocessing modules from keras
# we will import all layers and all models from Keras
# we import PIL to load images

from PIL import Image
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import * 
print('Imported Successfully!')

Imported Successfully!


In [13]:
# making a data generator for train data and test data
# we rescale the pixels between 0 and 1, because the pixels are between 0 and 255
# shear range determines the angle to which we rotate the images
# the zoom range determines the percentage of zoom

# we don't need to create more test images, as we want to know the labels only for these pictures
# we tranform the training images in order to have more training data and also to capture the features more accurately

train_datagen = image.ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = image.ImageDataGenerator(rescale = 1./255)

print('Created the Data Generator Objects.')

Created the Data Generator Objects.


In [14]:
# we create the training data by applying the generator to the training images
# we create the validdation data by applying the generator to the validation images 

train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/CovidDataset/Train',target_size=(224,224),batch_size=32, class_mode="binary")

val_generator = test_datagen.flow_from_directory('/content/gdrive/My Drive/CovidDataset/Val',target_size=(224,224),batch_size=32, class_mode="binary")


Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [15]:
# now we create the model and add layers to it
# the first layer is a convolutional layer, having a 3x3 kernel and the relu activation function
# also, being the first layer in the network, we have to give it the input shape
# in this case, we are using RGB images, so there are 3 channels, each one having 244x244 pixels
# we then add another conv layer with a kernel size 3x3
# these 2 layers are the equivalent of doing a single conv layer with the kernel size of 5x5
# but instead of having 25 params to train, now we have 9+9 = 18
# we reduce the size by applying a max pooling layer 
# then we use a dropout layer to avoid overfitting, by setting some of the neurons to 0
# we repeat this process and then we feed this to an ANN, by using the flatten layer to tranform 
# the data to one dimension
# the ANNS has a dense hidden layer with 64 neurons and the relu activation function
# then we set half of the neurons to 0
# we then add a dense layer with a single neuron because we a have a binary
# classification problem
# the last neuron has the activation function sigmoid, as we want the prob of an 
# image being covid or normal

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation="relu",input_shape=(224,224,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

# we compile the model with the adam optimizer

model.compile(loss="binary_crossentropy", optimizer="adam",metrics = ["accuracy"])

# we can see the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 54, 54, 128)      

In [16]:
# now we fit the model to the training data, using the validation data to validate the model

hist = model.fit(train_generator, epochs = 6, validation_data=val_generator, validation_steps=2)

Epoch 1/6
7/7 [==============================] - 80s 11s/step - loss: 2.1756 - accuracy: 0.5624 - val_loss: 0.6476 - val_accuracy: 0.5000
Epoch 2/6
7/7 [==============================] - 78s 11s/step - loss: 0.6641 - accuracy: 0.6288 - val_loss: 0.5607 - val_accuracy: 0.9000
Epoch 3/6
7/7 [==============================] - 76s 11s/step - loss: 0.4389 - accuracy: 0.8209 - val_loss: 0.2225 - val_accuracy: 0.9500
Epoch 4/6
7/7 [==============================] - 76s 11s/step - loss: 0.2915 - accuracy: 0.8870 - val_loss: 0.2222 - val_accuracy: 0.9333
Epoch 5/6
7/7 [==============================] - 76s 11s/step - loss: 0.2589 - accuracy: 0.9340 - val_loss: 0.1388 - val_accuracy: 0.9667
Epoch 6/6
7/7 [==============================] - 76s 11s/step - loss: 0.1857 - accuracy: 0.9355 - val_loss: 0.0837 - val_accuracy: 0.9667


In [ ]:
# We can see that after 6 epochs we got a validation accuracy of about 96.67%